In [1]:
%matplotlib qt
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from pyproj import CRS

In [2]:
gdf = gpd.read_file('/home/welling/geo/USA/all/tl_2021_us_county.shp')

In [3]:
gdf.columns

Index(['STATEFP', 'COUNTYFP', 'COUNTYNS', 'GEOID', 'NAME', 'NAMELSAD', 'LSAD',
       'CLASSFP', 'MTFCC', 'CSAFP', 'CBSAFP', 'METDIVFP', 'FUNCSTAT', 'ALAND',
       'AWATER', 'INTPTLAT', 'INTPTLON', 'geometry'],
      dtype='object')

In [4]:
gdf = gdf[gdf.STATEFP=='06']  # select CA

In [5]:
gdf.head()

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
8,06,091,00277310,06091,Sierra,Sierra County,06,H1,G4020,None,None,None,A,2468694582,23299110,+39.5769252,-120.5219926,"POLYGON ((-120.55587 39.50874, -120.55614 39.5..."
325,06,067,00277298,06067,Sacramento,Sacramento County,06,H1,G4020,472,40900,None,A,2500045887,75340560,+38.4501363,-121.3443291,"POLYGON ((-121.43991 38.25553, -121.44002 38.2..."
329,06,083,00277306,06083,Santa Barbara,Santa Barbara County,06,H1,G4020,None,42200,None,A,7080860229,2729213507,+34.5366774,-120.0383645,"MULTIPOLYGON (((-120.58226 34.10752, -120.5790..."
346,06,009,01675885,06009,Calaveras,Calaveras County,06,H1,G4020,None,None,None,A,2641837359,43789489,+38.1910682,-120.5541065,"POLYGON ((-120.37620 38.14265, -120.37616 38.1..."
394,06,111,00277320,06111,Ventura,Ventura County,06,H1,G4020,348,37100,None,A,4767622161,947345735,+34.3587477,-119.1331453,"MULTIPOLYGON (((-119.63607 33.28071, -119.6347..."


In [6]:
def add_area_and_label_coords(some_gdf):
    some_gdf['area'] = some_gdf.area
    some_gdf['coords'] = some_gdf['geometry'].apply(lambda x: x.representative_point().coords[:])
    some_gdf['coords'] = [coords[0] for coords in some_gdf['coords']]

In [7]:
def plot_with_labels(some_gdf, ax, name_col=None, field_col=None):
    if field_col is None:
        some_gdf.plot(ax=ax)
    else:
        some_gdf.plot(column=field_col, ax=ax, legend=True)
    if name_col is not None:
        for idx, row in some_gdf.iterrows():
            ax.annotate(text=row[name_col], xy=row['coords'],
                         horizontalalignment='center')

In [8]:
def calc_overall_centroid(some_gdf):
    """
    Use the 'coords' and 'area' columns to estimate an overall centroid
    """
    sum_x = 0.0
    sum_y = 0.0
    sum_area = 0.0
    for idx, row in some_gdf.iterrows():
        coord_x, coord_y = row['coords']
        sum_x += coord_x * row['area']
        sum_y += coord_y * row['area']
        sum_area += row['area']
    centroid_x = sum_x / sum_area
    centroid_y = sum_y / sum_area
    return centroid_x, centroid_y

In [9]:
add_area_and_label_coords(gdf)
centroid_x, centroid_y = calc_overall_centroid(gdf)
print(centroid_x, centroid_y)

-119.66034006773158 37.21896331381407


/tmp/ipykernel_19992/4182452352.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  some_gdf['area'] = some_gdf.area


In [10]:
fig, axes = plt.subplots(1,2)
plot_with_labels(gdf, field_col='area', name_col='NAME', ax=axes[0])
plot_with_labels(gdf, ax=axes[1])

In [11]:
gdf.crs


<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - onshore and offshore: Canada - Alberta; British Columbia; Manitoba; New Brunswick; Newfoundland and Labrador; Northwest Territories; Nova Scotia; Nunavut; Ontario; Prince Edward Island; Quebec; Saskatchewan; Yukon. Puerto Rico. United States (USA) - Alabama; Alaska; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Hawaii; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming. US Virgin Islands. British Virgin Islands

# Orthographic projection centered on our centroid
See [Orthographic projection with pyproj for penguin tracking in Antarctica](https://towardsdatascience.com/orthographic-projection-with-pyproj-for-penguin-tracking-in-antarctica-18cd2bf2d570) for the trick.

In [12]:
lat = centroid_y
lon = centroid_x
ortho = CRS.from_proj4("+proj=ortho +lat_0={} +lon_0={} +x_0=0 +y_0=0 +a=6370997 +b=6370997 +units=m +no_defs".format(lat, lon))

In [13]:
ortho_gdf = gdf.to_crs(ortho)
ortho_gdf.crs

<Derived Projected CRS: +proj=ortho +lat_0=37.21896331381407 +lon_0=-119.6 ...>
Name: unknown
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unknown
- method: Orthographic
Datum: unknown
- Ellipsoid: Normal Sphere (r=6370997)
- Prime Meridian: Greenwich

In [14]:
utm_proj = CRS.from_proj4("+proj=utm +zone=10 +north")
print(utm_proj)

+proj=utm +zone=10 +north +type=crs


In [15]:
utm_gdf = gdf.to_crs(utm_proj)
utm_gdf.crs

<Derived Projected CRS: +proj=utm +zone=10 +north +type=crs>
Name: unknown
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: UTM zone 10N
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [16]:
utm_gdf.plot()

<AxesSubplot:>

In [17]:
fig, axes = plt.subplots(1,2)
plot_with_labels(gdf, ax=axes[0])
plot_with_labels(ortho_gdf, ax=axes[1])

In [18]:
new_gdf = gdf.to_crs("EPSG:32633")

In [19]:
new_gdf.crs

<Derived Projected CRS: EPSG:32633>
Name: WGS 84 / UTM zone 33N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 12°E and 18°E, northern hemisphere between equator and 84°N, onshore and offshore. Austria. Bosnia and Herzegovina. Cameroon. Central African Republic. Chad. Congo. Croatia. Czechia. Democratic Republic of the Congo (Zaire). Gabon. Germany. Hungary. Italy. Libya. Malta. Niger. Nigeria. Norway. Poland. San Marino. Slovakia. Slovenia. Svalbard. Sweden. Vatican City State.
- bounds: (12.0, 0.0, 18.0, 84.0)
Coordinate Operation:
- name: UTM zone 33N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [20]:
fig, axes = plt.subplots(1,2)
plot_with_labels(gdf, ax=axes[0])
plot_with_labels(new_gdf, ax=axes[1])

In [21]:
gdf = gpd.read_file('/home/welling/geo/USA/all/tl_2021_us_county.shp')
display(gdf.columns)
gdf = gdf[gdf.STATEFP=='42']  # select PA
add_area_and_label_coords(gdf)

Index(['STATEFP', 'COUNTYFP', 'COUNTYNS', 'GEOID', 'NAME', 'NAMELSAD', 'LSAD',
       'CLASSFP', 'MTFCC', 'CSAFP', 'CBSAFP', 'METDIVFP', 'FUNCSTAT', 'ALAND',
       'AWATER', 'INTPTLAT', 'INTPTLON', 'geometry'],
      dtype='object')

/tmp/ipykernel_19992/4182452352.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  some_gdf['area'] = some_gdf.area


In [22]:
def build_fips_string(row):
    return f"{row['STATEFP']}{row['COUNTYFP']}"
gdf['fips_string'] = gdf.apply(build_fips_string, axis=1)
gdf.head()

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,...,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,area,coords,fips_string
19,42,007,01214112,42007,Beaver,Beaver County,06,H1,G4020,430,...,None,A,1125854819,24162295,+40.6841401,-080.3507209,"POLYGON ((-80.27803 40.53674, -80.29151 40.527...",0.122502,"(-80.33356401630434, 40.6665865)",42007
47,42,053,01213669,42053,Forest,Forest County,06,H1,G4020,None,...,None,A,1106595595,8369675,+41.5133040,-079.2497050,"POLYGON ((-78.95860 41.52549, -78.95861 41.525...",0.120257,"(-79.23552915941254, 41.475388499999994)",42053
95,42,117,01209189,42117,Tioga,Tioga County,06,H1,G4020,None,...,None,A,2936765818,8279717,+41.7668593,-077.2572881,"POLYGON ((-77.21159 41.54545, -77.21177 41.545...",0.318905,"(-77.25034505294256, 41.771712)",42117
108,42,043,01213667,42043,Dauphin,Dauphin County,06,H1,G4020,276,...,None,A,1359407363,86209037,+40.4125646,-076.7926343,"POLYGON ((-76.59217 40.25428, -76.59214 40.254...",0.153387,"(-76.83720556170019, 40.389572)",42043
165,42,127,01213692,42127,Wayne,Wayne County,06,H1,G4020,None,...,None,A,1879734030,64785676,+41.6466021,-075.2924932,"POLYGON ((-75.29834 41.36740, -75.29849 41.367...",0.210166,"(-75.26130220558844, 41.616035999999994)",42127


In [23]:
fig, axes = plt.subplots(2,1)
plot_with_labels(gdf, name_col='NAME', ax=axes[0])
plot_with_labels(gdf, name_col='fips_string', ax=axes[1])